# Import the Libraries Needed

In [6]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
import requests

r  = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

data = r.text

soup = BeautifulSoup(data, 'html.parser')

# Go Grab the table and pull the pieces out with Soup

Find the useful table
Work through each row
    - If the row has Not Assigned as a Borough - move on
    - If the row has no Neighborhood but a Borough - copy the Neighborhood
Save the dict into a dataframe

Now we have a dataframe but multiple rows per PostalCode and Borough

In [8]:
table = soup.find( "table", {"class":"wikitable sortable"} )

td = table.findAll('tr')[1:]

hoods = []

for data in td:
    col = data.find_all('td')
    details = {}
    for i,col in enumerate(col):
        if i == 1:
            details['Borough'] = (col.text.replace('\n',''))
            if details['Borough'] == 'Not assigned':
                break
        if i == 0:
            details['PostalCode'] = (col.text.replace('\n',''))
        if i == 2: 
            details['Neighborhood'] = (col.text.replace('\n',''))
            if details['Neighborhood'] == 'Not assigned':
                details['Neighborhood'] = details['Borough']
            hoods.append(details)

df=pd.DataFrame(hoods)
df=df[['PostalCode','Borough','Neighborhood']]
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


# Use Group by to do the work of concatinating the like Boroughs together

In [9]:
dfn=df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(','.join).reset_index()
dfn.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


# Displaying the Shape of the Final Dataset

In [10]:
dfn.shape

(103, 3)

# Adding GEO data

Dowload spreadsheet into dataframe
The URL is dynamic and needs to be updated daily

In [12]:
geo_data=pd.read_csv('https://dl.boxcloud.com/d/1/b1!2lc7tg9l4YiRIXBCc-QdycOyiD_FIYAsOd26x3wFGTyqbbXdnUrX289dnA2uiT-45wE2VvzguVqOeSObWZuiP8-aNGyfublsgfeATMzEnLxc6rQIRpC8jHY7kZcyOWXskLXf6Ov8RRDlw8SvBYwneLF-32VAXJxBFIKBUatXnldl9xKiNTmfpdXonkTo18NVkj97-PxHehLcRq2wOcidVg_Crzz2Fobhecf7DbUIC8HhY4T1LJHKEpmVMW3VMGZziHhUGhh8z_G17t38Xamc_tofFvnF9tuPWtLKOdwKfLo9e8Khk1oWeq721PRJR_rH-dqnzeZHPySMbgRgKOhz_9ZUjfQZQg3x7IyMaJsJZfvx3w3pfSMFbOEvFUeJyu6yhM9bpBpRdwi8CP4jUM5S41IWFvQ576EecnpIBZ90kWtmYFqNWCHJZHDG2dUSSAUPHa01ZJEdHooDFpzpYkpiE1ARLkTrGWXrO_jj3zY8MS5mPsMbaDq7G4Mwi2kUggswbL10Jst37L12xBBthLqCsJbn10rjrpWtHmnvrNWoD_yyzwxIhEZlDfjN1qOOng4LFUfIy1BztL3JzKmbBY8i6_OPJ_0wkCJ4QEWJWb4zp2kzpQ2m3YnPqPXSRFszeosPPTC8fBJLopLmM32CWT_G-l9l7HlTA0Lw7sFRFG0HtikMx4yA5lXZe34t3veB36u2KpHJtNjVAnPkgQj3zcngPy1th38JRG0_fPFYmIVC9QNazMQBn1CSqR-aqQopHkVJNkUVI2eA1JY1Mj1iwqTaNuUqS12aHVzqgtEkTn6BRKSqWRKY_uHa_NJTuWkVCgk7yMswEsPLiAJlb_xbY3dMn2tIVZKE7rPtU9baM4ZpTydaGXEAdCVpyJqXmE7-KPjCIVT7qqzu75slxphLrCsTt9HPG9EgEfk2PY1WSz0K0w2OUDoCNiwTK5bvBvUlx7iwuEnCGFnkZzx5R2jgpErF-uvxoZinnr6ArywDrKESukR5keHPERvH1v84Hw3dQZ2dXBHVi7_YL7hp65ZsFbHO1egi3V4GqDcQ7knyNWl7PTPxB73YOOCXA47cR4JZHpx_Ea48E56OLtxV71FCyfkO7apYOlWlhPzCGfwO3Mm3XXgIxd-O4zNgu5yx-dOtLZmGez5eGjs2-7qzPc9Bf46OIMor7uTRNNAHhMggyMW8Zw0MwJhrQ_rwCcthQgPhfdpSwhKfLtIE7n2kUxPXbBsIZkd0h6eLxwI3dvvkvqfXb8bG0gpdXOlqk7EtsochJxK0knkWiulIQ9VPtJpLZ_y9VYDzcp1XZy_a8Heukx3v1HC0PWpjWUU4GHaW703qz8uKg-HAreMx76jtycqkaWqJ5ymeTBLMRQ6VFxb7ZL9yeqLkafr8e9txCGtMBZSb/download')
print(geo_data.shape)
geo_data.head()

(103, 3)


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


# Using the SQL like Pandas MERGE feature
Join the two datasets together

Keep all rows from the Wikipedia Page and bring in the locations from the loaded CSV

In [14]:
alldata=pd.merge(dfn, geo_data, how='left', left_on=['PostalCode'], right_on=['Postal Code'],
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)
print(alldata.shape)
alldata.head()

(103, 6)


,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476
